# Youtube spam detection

Download [link](http://www.dt.fee.unicamp.br/~tiago//youtubespamcollection/)

In [1]:
from fastai2.text.all import *
from lazylabel.text.all import *

In [2]:
source = Path('/home/lgvaz/.data/youtube_spam/')

In [3]:
fns = get_files(source, extensions='.csv')
dfs = [pd.read_csv(fn) for fn in fns]

In [4]:
df = pd.concat(dfs)
print(df.size); df.head()

9780


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z12pgdhovmrktzm3i23es5d5junftft3f,lekanaVEVO1,2014-07-22T15:27:50,i love this so much. AND also I Generate Free Leads on Auto Pilot &amp; You Can Too! http://www.MyLeaderGate.com/moretraffic﻿,1
1,z13yx345uxepetggz04ci5rjcxeohzlrtf4,Pyunghee,2014-07-27T01:57:16,http://www.billboard.com/articles/columns/pop-shop/6174122/fan-army-face-off-round-3 Vote for SONES please....we're against vips....please help us.. &gt;.&lt;﻿,1
2,z12lsjvi3wa5x1vwh04cibeaqnzrevxajw00k,Erica Ross,2014-07-27T02:51:43,"Hey guys! Please join me in my fight to help abused/mistreated animals! All fund will go to helping pay for vet bills/and or helping them find homes! I will place an extra emphasis on helping disabled animals, ones otherwise would just be put to sleep by other animal organizations. Donate please. http://www.gofundme.com/Angels-n-Wingz﻿",1
3,z13jcjuovxbwfr0ge04cev2ipsjdfdurwck,Aviel Haimov,2014-08-01T12:27:48,http://psnboss.com/?ref=2tGgp3pV6L this is the song﻿,1
4,z13qybua2yfydzxzj04cgfpqdt2syfx53ms0k,John Bello,2014-08-01T21:04:03,Hey everyone. Watch this trailer!!!!!!!! http://believemefilm.com?hlr=h2hQBUVB﻿,1


In [5]:
df['original'] = df['CONTENT']

In [6]:
cder,tkzer,nmzer = mk_transform(ColReader('original')),Tokenizer.from_df('CONTENT'),Numericalize()

In [7]:
ABSTAIN,SPAM,HAM = 'abstain','spam','ham'
vocab = [ABSTAIN,SPAM,HAM]

In [8]:
labeller = Labeller(vocab)
@labeller(tkzer)
def check(x): return SPAM if 'check' in x else ABSTAIN
@labeller(cder)
def check_out(x): return SPAM if 'check out' in x.lower() else ABSTAIN

In [9]:
tls = TfmdLists(df, [cder,tkzer,nmzer])
tasks = tasks_labels(tls, vocab)

In [10]:
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [11]:
labeller.summary(dls.train)

,Coverage,Polarity
check_out,20.68% (397),1
check,24.11% (463),1


In [12]:
def get_sentences(tls, idxs):
    sentences = {}
    for i in idxs:
        tks = tls.decode(tls[i])
        sentences[i] = (' '.join(decode_spec_tokens(tks.split(' '))))
    return pd.DataFrame(sentences.values(), sentences.keys())

In [13]:
idxs = labeller.find(dls.train, ['check', 'check_out'], [SPAM, ABSTAIN])

In [14]:
get_sentences(tls, idxs).sample(10)

,0
861,xxbos check men out i put xxunk of xxunk into my music but xxunk not many watch it﻿
802,"xxbos Just coming to check if people are still xxunk this video . And xxunk , they still xxunk"
748,xxbos i do nt even watch it anymore i just come here to check on 2 Billion or xxunk
756,xxbos Check my channel please ! And listen to the best music ever : p﻿
691,"xxbos Hey Music Fans i really appreciate all of you who take time to read this , and check my music out ! I&'m just a 15 year old boy DREAMING of being a successful MUSICIAN in the music world . i do lots of covers , and piano covers . But i do nt have money to advertise . a simple thumbs up to my comment , a comment on my videos or a SUBSCRIPTION would be a step forward ! It will only be a few seconds of your life that u won&'t regret !!! Thank u to all the people who just give me a chance it means a lot ! :)"
1410,xxbos ***** PLEASE READ 55555 ***** Hey everyone ! I&'m a 19 year old student who loves to sing . i record and upload covers on my channel . i would love if you could check them out . Just give me a chance . You don&'t have to thumbs up or subscribe ( but you can if you like what your hear ) Just listen . Xxunk really appreciate . xxunk YOU for your time . ~
34,"xxbos Hello ! I 'm Xxunk , I 'm a singer from Xxunk ! i was part of a boy - girl band named xxunk , and I 'm now singing on my own xxunk i wanted to play my own pop and pop - rock songs . It would mean a lot if you could have a look at my channel to check my music and watch my new video ! ! and if u like , subscribe to it ! XOXO THANKS ! ! XXUNK : if you like a lot my channel , you can share it with your friends ! ! Haha ! ! LOL XXUNK"
294,xxbos Such a good song xxunk Katy sounds great over the melody . I m xxunk as an artist everyday from my hit song ' crazy ' which has got my name out there . ca nt thank my fans more for their support . If you could take a moment ? to check it and my music ? maybe you 'll join me to make my dream come true :) thank you for your time﻿
751,"xxbos Came here to check the views , xxunk"
678,"xxbos Hey Music Fans i really appreciate any of you who will take the time to read this , and check my music out ! I&'m just a 15 year old boy DREAMING of being a successful MUSICIAN in the music world . i do lots of covers , and piano covers . But i don&'t have money to advertise . a simple thumbs up to my comment , a comment on my videos or a SUBSCRIPTION would be a step forward ! It will only be a few seconds of your life that you won&'t regret !!! Thank u to all the people who just give me a chance ! :)"


In [15]:
@labeller(cder)
def regex_check_out(x):
    return SPAM if re.search(r'check.*out', x, flags=re.I) else ABSTAIN

In [16]:
@labeller(cder)
def regex_check(x):
    return SPAM if re.search(r'(check\s(my|this))|check\s(\w+)?\s?out', x, flags=re.I) else ABSTAIN

In [17]:
tasks = tasks_labels(tls, vocab)
dset = Datasets(tls=[tasks], tfms=[])
dls = dset.dataloaders()

In [18]:
labeller.summary(dls.train)

,Coverage,Polarity
check_out,20.73% (398),1
regex_check_out,22.71% (436),1
regex_check,23.28% (447),1
check,24.11% (463),1


In [19]:
idxs = labeller.find(dls.train, ['regex_check', 'regex_check_out'], [SPAM, ABSTAIN])
print(len(idxs))
get_sentences(tls, idxs).sample(10)

17


,0
940,"xxbos I m a RAPPER / SONGWRITER , check my video please .. also subscribe for more thanks :) tell me what you xxunk"
1329,xxbos ┏ ━━━ ┓┓┓ ┏┏┏ ┓┓┓ ╋╋╋ ┏┏┏ ┓┓┓ ┏┏┏ 333 ━━━ ┓┓┓ ┏┏┏ 333 ━━━ ┓┓┓ ┏┏┏ ┓┓┓ ╋╋╋ ╋╋╋ ┏┏┏ ┓┓┓ ┃┃┃ ┏┏┏ ━━━ ┓┓┓ 333 ┃┃┃ ╋╋╋ 333 ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┗┗┗ ┓┓┓ ┏┏┏ ┓┓┓ ┃┃┃ ┃┃┃ ┗┗┗ ┓┓┓ ┏┏┏ ┛┛┛ ┃┃┃ ┃┃┃ ┗┗┗ ━━━ ━━━ ┓┓┓ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ 333 ┃┃┃ ╋╋╋ ┃┃┃ ┃┃┃ ╋╋╋ 444 ┃┃┃ ┗┗┗ ┓┓┓ ┗┗┗ ┛┛┛ ┏┏┏ ┗┗┗ ━━━ ━━━ ┓┓┓ ┃┃┃ ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ ┃┃┃ ╋╋╋ 444 ┃┃┃ ╋╋╋ ┗┗┗ ┓┓┓ ┏┏┏ ┛┛┛ ┃┃┃ ┗┗┗ ━━━ ┛┛┛ 333 ┃┃┃ ╋╋╋ 333 ┃┃┃ ┏┏┏ ━━━ ┓┓┓ ┃┃┃ ┏┏┏ ┛┛┛ ┗┗┗ ┛┛┛ ┃┃┃ ╋╋╋ ╋╋╋ ┃┃┃ ┃┃┃ ┗┗┗ 333 ━━━ ┛┛┛ ┗┗┗ ┛┛┛ ╋╋╋ ┗┗┗ ┛┛┛ ┗┗┗ ┛┛┛ ╋╋╋ ┗┗┗ ┛┛┛ ┗┗┗ 333 ━━━ ┛┛┛ ╋╋╋ ╋╋╋ ┗┗┗ ┛┛┛ CHECK MY VIDEOS AND SUBSCRIBE AND LIKE XXUNK
1820,"xxbos Need money ? check my channel and subscribe , soon will post how to get it ) ﻿"
804,xxbos Check my channel﻿
52,"xxbos Hey guys go to check my video name "" xxunk my xxunk"
209,"xxbos Rap from Xxunk , check my xxunk"
25,xxbos Hi Guys ! check this awesome EDM & & House mix :) thanks a lot .. https : / / soundcloud.com / xxunk / edm - house - xxunk
34,"xxbos Hello ! I 'm Xxunk , I 'm a singer from Xxunk ! i was part of a boy - girl band named xxunk , and I 'm now singing on my own xxunk i wanted to play my own pop and pop - rock songs . It would mean a lot if you could have a look at my channel to check my music and watch my new video ! ! and if u like , subscribe to it ! XOXO THANKS ! ! XXUNK : if you like a lot my channel , you can share it with your friends ! ! Haha ! ! LOL XXUNK"
728,xxbos You should check my channel for Funny XXUNK
764,"xxbos if you like xxunk talent , xxunk lyrics , xxunk real hip hop Everyone check my newest sound Xxunk x - Got the Xxunk ( prod by . Xxunk the Model Xxunk ) COMMENT TELL ME WHAT YOU THINK DO nt BE XXUNK !!!! ---- 1111 //// 7777 Xxunk"


In [20]:
idxs = labeller.find(dls.train, ['regex_check'], [SPAM])
get_sentences(tls, idxs).sample(10)

,0
1766,xxbos Check out this video on youtube:﻿
25,xxbos Hi Guys ! check this awesome EDM & & House mix :) thanks a lot .. https : / / soundcloud.com / xxunk / edm - house - xxunk
1789,xxbos Check out this video on youtube:﻿
546,"xxbos You guys should check out this EXTRAORDINARY website called ZONEPA.COM . You can make money online and start working from home today as i am ! i am making over $ 3 , 000 +++ perperper monthmonthmonth atatat ZONEPA.COM ! Visit Zonepa.com and check it out ! How does the mother xxunk the xxunk xxunk ? The fear xxunk the xxunk . When does the space xxunk the xxunk shame ?"
1748,xxbos Check out this video on youtube:﻿
372,xxbos Check out this playlist on youtube:﻿
711,xxbos * * check OUT MY NEW MIXTAPE **** **** **** checkcheckcheckcheck OUT MY NEW MIXTAPE 4444 **** **** **** checkcheckcheckcheck OUT MY NEW MIXTAPE 3333 **** 3333 **** CHECK OUT MY NEW MIXTAPE 6666 **** CHECK OUT MY NEW MIXTAPE 4444 **** **** **** checkcheckcheckcheck OUT MY NEW MIXTAPE 4444 **** **** **** checkcheckcheckcheck OUT MY NEW MIXTAPE 3333 **** 3333 **** CHECK OUT MY NEW MIXTAPE 6666 **** CHECK OUT MY NEW MIXTAPE 4444 **** **** **** checkcheckcheckcheck OUT MY NEW MIXTAPE 4444 **** **** **** checkcheckcheckcheck OUT MY NEW MIXTAPE 3333 **** 3333 **** CHECK OUT MY NEW MIXTAPE 6...
1901,xxbos Check out this video on youtube : xxunk
1031,xxbos Please check out my xxunk xxunk
820,"xxbos My videos are half way decent , check them out if you xxunk"


If instead of having a recursive modification of __call__ we adopt a list of listeners, we can generate all stuff for the preprocessor first and then pass wherever it's necessary. This would use a for loop

In [ ]:
@labeller(cder, TextBlob)
def tb_polarity(x, tb):
    return HAM if tb.polarity > 0.9 else ABSTAIN
@labeller(cder, TextBlock)
def tb_sentiment(x, tb):
    return HAM if tb.subjectivity >= 0.5 else ABSTAIN

In [21]:
from textblob import TextBlob

In [22]:
text = df.iloc[1291]['original']
print(text)
scores = TextBlob(text)

You guys should check out this EXTRAORDINARY website called ZONEPA.COM . You can make money online and start working from home today as I am! I am making over $3,000+ per month at ZONEPA.COM ! Visit Zonepa.com and check it out! How does the mammoth waste achieve the shock? How does the limit reduce the delicate minute? How does the meaty scale adapt the oil?


In [23]:
scores.polarity, scores.subjectivity

(0.050347222222222175, 0.6333333333333333)